In [1]:
from transformer_chat_model.chat_models import TransformerChatModel

In [2]:
from transformers import BitsAndBytesConfig
import torch
from langchain_core.tools import tool

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, AIMessageChunk

In [4]:
pretrained_model_name_or_path = "Qwen/Qwen2.5-3B-Instruct"
# Create your quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
chat_model = TransformerChatModel(
    pretrained_model_name_or_path=pretrained_model_name_or_path,
    quantization_config = bnb_config,
    max_new_tokens = 1024,
    device = "auto"
    )

Loading Model ...



Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

In [5]:
@tool
def check_weather(location:str) -> str:
    """Check weather at a location"""
    return "Rain" if location == "Hannoi" else "Cloudy"
@tool
def factorial(num:int)->int:
    """Calculate fatorial of number"""
    return 1

In [7]:
res = []
for i in chat_model.stream("Hello, how are you?"):
    res.append(i)
print(res)

[AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4', tool_calls=[], invalid_tool_calls=[], tool_call_chunks=[]), AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4', tool_calls=[], invalid_tool_calls=[], tool_call_chunks=[]), AIMessageChunk(content='Hello! ', additional_kwargs={}, response_metadata={}, id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4', tool_calls=[], invalid_tool_calls=[], tool_call_chunks=[]), AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4', tool_calls=[], invalid_tool_calls=[], tool_call_chunks=[]), AIMessageChunk(content="I'm ", additional_kwargs={}, response_metadata={}, id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4', tool_calls=[], invalid_tool_calls=[], tool_call_chunks=[]), AIMessageChunk(content='an ', additional_kwargs={}, response_metadata={}, id='

In [8]:
for r in res:
    print(r)

content='' additional_kwargs={} response_metadata={} id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content='' additional_kwargs={} response_metadata={} id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content='Hello! ' additional_kwargs={} response_metadata={} id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content='' additional_kwargs={} response_metadata={} id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content="I'm " additional_kwargs={} response_metadata={} id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content='an ' additional_kwargs={} response_metadata={} id='lc_run--019c721a-d1e8-7101-8815-a49f3d137de4' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content='artificial ' additional